# AI Matchmaking Deep Learning Prediction Model

USER INPUT AND MATCHMAKING PREDICTION

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import auc, plot_precision_recall_curve
import keras
from keras.datasets import mnist
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from keras.optimizers import rmsprop_v2
import pylab as plt
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/compathon/pawpals_features_without_test.csv",na_values="?")
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Age1,Gender1,Breed1,Vaccination1,Age2,Gender2,Breed2,Vaccination2
0,10,1,1,1,12,0,1,1
1,24,0,0,1,20,1,0,1
2,5,0,2,1,8,1,2,1
3,14,1,3,1,20,0,3,1
4,15,1,1,1,11,0,1,1


In [ ]:
#Uisng Label Encoder
data_string = data.dtypes==object
data_cols = data.columns[data_string].tolist()
le = LabelEncoder()
data[data_cols] = data[data_cols].apply(lambda col: le.fit_transform(col))
data.head()

,Age1,Gender1,Breed1,Vaccination1,Age2,Gender2,Breed2,Vaccination2,Match
0,10,1,1,1,12,0,1,1,1
1,24,0,0,1,20,1,0,1,1
2,5,0,2,1,8,1,2,1,0
3,14,1,3,1,20,0,3,1,1
4,15,1,1,1,11,0,1,1,1


In [ ]:
X = data.iloc[:, 0:-1].values
y = data.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )

class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

[[10  1  1  1 12  0  1  1]
 [24  0  0  1 20  1  0  1]
 [ 5  0  2  1  8  1  2  1]
 [14  1  3  1 20  0  3  1]
 [15  1  1  1 11  0  1  1]
 [18  1  0  1 21  1  1  1]
 [ 6  0  2  1 24  0  2  1]
 [17  1  4  1 15  0  4  1]
 [ 5  0  4  1 10  1  4  1]
 [24  1  2  1 20  0  2  0]
 [32  0  0  1 28  1  0  1]
 [25  1  1  1 21  0  2  1]
 [10  1  0  1 15  0  0  1]
 [12  0  1  1 17  1  1  1]
 [16  0  1  1 20  1  1  1]
 [20  0  2  1 16  1  2  1]
 [13  1  3  1 10  0  3  1]
 [12  0  3  1 16  1  0  1]
 [11  1  4  1 14  1  3  1]
 [ 9  1  4  0  4  0  4  1]
 [ 8  1  0  1  2  0  0  1]
 [ 5  0  4  1  7  1  4  1]
 [72  1  1  1 31  0  1  1]
 [61  0  2  1 57  1  2  1]
 [48  1  0  1 25  0  0  1]
 [12  0  2  1 16  1  2  1]
 [44  1  3  1 40  0  3  1]
 [71  1  1  1 67  0  1  1]
 [54  0  2  1 50  1  2  1]
 [32  1  0  1 28  0  0  1]
 [ 5  0  3  1  1  1  3  0]
 [71  0  4  1 10  1  4  1]
 [63  1  4  1 59  0  4  1]
 [15  1  4  1 18  0  4  1]
 [65  0  2  1 60  0  2  1]
 [12  0  3  1  8  0  3  1]
 [21  1  2  1  2  0  2  1]
 

{0: 1.58, 1: 0.7314814814814815}

In [ ]:
#Artificial Neural Network

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)


# ann.save('pawpals_model.sav')
pickle.dump(ann, open('pawpals_model.pkl', 'wb'))

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/24
7/7 [==============================] - 1s 4ms/step - loss: 0.6956 - accuracy: 0.6835
Epoch 2/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6903 - accuracy: 0.6329
Epoch 3/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6868 - accuracy: 0.5696
Epoch 4/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6845 - accuracy: 0.5949
Epoch 5/24
7/7 [==============================] - 0s 5ms/step - loss: 0.6796 - accuracy: 0.6329
Epoch 6/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6751 - accuracy: 0.6203
Epoch 7/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6696 - accuracy: 0.5570
Epoch 8/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6604 - accuracy: 0.6582
Epoch 9/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6528 - accuracy: 0.7342
Epoch 10/24
7/7 [==============================] - 0s 4ms/step - loss: 0.6431 - accuracy: 0.7722
Epoch 11/24
7/7 [======================

#Model Prediction

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/compathon/pawpals_features_without_test.csv",na_values="?")
data.head()

,Age1,Gender1,Breed1,Vaccination1,Age2,Gender2,Breed2,Vaccination2
0,10,1,1,1,12,0,1,1
1,24,0,0,1,20,1,0,1
2,5,0,2,1,8,1,2,1
3,14,1,3,1,20,0,3,1
4,15,1,1,1,11,0,1,1


In [ ]:
import pickle

filename = 'pawpals_model.pkl'

data_new = pd.read_csv (r'/content/drive/MyDrive/compathon/pawpals_features_without.csv')
data_new = data_new.sample(frac=1).reset_index(drop=True)
data_new.head()

X = data_new.values
sc = StandardScaler()
X = sc.fit_transform(X)

loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X)
y_pred = (y_pred > 0.5)

data_new["Prediction"] = y_pred
data_new.replace({False: "No Match", True: "Match"}, inplace=True)
data_new.to_csv('result.csv', index = False, encoding='utf-8')

data_new

,Age1,Gender1,Breed1,Vaccination1,Age2,Gender2,Breed2,Vaccination2,Prediction
0,21,1,2,1,2,0,2,1,No Match
1,16,0,1,1,20,1,1,1,Match
2,18,0,0,1,13,1,0,1,Match
3,11,0,1,1,7,1,1,1,Match
4,6,0,2,1,24,0,2,1,No Match
...,...,...,...,...,...,...,...,...,...
94,12,0,3,1,16,1,3,1,No Match
95,25,0,1,1,21,1,1,1,Match
96,63,1,4,1,59,0,4,1,No Match
97,72,1,1,1,31,0,1,1,No Match
